<a href="https://colab.research.google.com/github/joseluisll/AnalisisIbex/blob/main/AnalisisIbex.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
# Instala plotly en Colab
!pip install plotly
!pip install gradio
!pip install schedule==1.1.0

import numpy as np
import pandas as pd
import plotly.graph_objects as go

**Creamos el vector de tiempos, para todos el mismo, cada posicion es un día.**

In [12]:
vector_fechas_test = pd.date_range(start='2025-01-01', end='2025-04-11')

In [53]:
vector_fechas = pd.date_range(start='2024-01-01', end='2025-04-11')

#Preparar la serie HISTORICA de IBEX35 desde 1/1/2015 A 11/04/2025

In [54]:
# Abrir la serie de IBEX35
df_ibex = pd.read_csv('datos/datos_historicos_IBEX35.csv', parse_dates=['Fecha'], index_col='Fecha')
# Convert 'Último' column to float
df_ibex['Último'] = df_ibex['Último'].apply(lambda x: float(x.replace('.', '').replace(',', '.')))


In [55]:
# Crear un dataframe con índice vector_fechas y valores en cero
df_ibex_reindexado = pd.DataFrame(index=vector_fechas, data=0, columns=df_ibex.columns)
df_ibex_reindexado = df_ibex_reindexado.astype(object)

# Copiar los valores de df_ibex a df_ibex_reindexado cuando los índices coinciden
for fecha in df_ibex.index:
  if fecha in df_ibex_reindexado.index:
    df_ibex_reindexado.loc[fecha] = df_ibex.loc[fecha]



# Method 1: Min-Max Scaling (Normalization to [0, 1])
df_ibex_reindexado['Normalizado'] = (df_ibex_reindexado['Último'] - df_ibex_reindexado['Último'].min()) / (df_ibex_reindexado['Último'].max() - df_ibex_reindexado['Último'].min())

#Preparar la serie HISTORICA de Petroleo_WTI desde 1/1/2015 A 11/04/2025

In [56]:
#datos históricos de petroleo_WTI y la normalizo
df_petroleo = pd.read_csv('datos/datos_historico_petroleo_wti.csv', parse_dates=['Fecha'], index_col='Fecha')




In [57]:
# Convert 'Último' column to float
df_petroleo['Último'] = df_petroleo['Último'].apply(lambda x: float(x.replace('.', '').replace(',', '.')))

In [58]:
df_petroleo_reindexado = pd.DataFrame(index=vector_fechas, data=0, columns=df_petroleo.columns)
df_petroleo_reindexado = df_petroleo_reindexado.astype(object)

# Copiar los valores de df_petroleo a df_petroleo_reindexado cuando los índices coinciden
for fecha in df_petroleo.index:
  if fecha in df_petroleo_reindexado.index:
    df_petroleo_reindexado.loc[fecha] = df_petroleo.loc[fecha]


# Min-Max Scaling
df_petroleo_reindexado['Normalizado'] = (df_petroleo_reindexado['Último'] - df_petroleo_reindexado['Último'].min()) / (df_petroleo_reindexado['Último'].max() - df_petroleo_reindexado['Último'].min())


**PREPARAR LA SERIE HISTÓRICA DE VOLATILIDAD DEL VIX MISMA ESCALA TEMPORAL QUE LAS ANTERIORES**

In [59]:
# Abrir la serie de IBEX35
df_vix = pd.read_csv('datos/datos_historicos_VIX.csv', parse_dates=['Fecha'], index_col='Fecha')

# Convert 'Último' column to float
df_vix['Último'] = df_vix['Último'].apply(lambda x: float(x.replace('.', '').replace(',', '.')))

df_vix_reindexado = pd.DataFrame(index=vector_fechas, data=0, columns=df_vix.columns)
df_vix_reindexado = df_vix_reindexado.astype(object)

# Copiar los valores de df_petroleo a df_petroleo_reindexado cuando los índices coinciden
for fecha in df_vix.index:
  if fecha in df_petroleo_reindexado.index:
    df_vix_reindexado.loc[fecha] = df_vix.loc[fecha]


# Min-Max Scaling
df_vix_reindexado['Normalizado'] = (df_vix_reindexado['Último'] - df_vix_reindexado['Último'].min()) / (df_vix_reindexado['Último'].max() - df_vix_reindexado['Último'].min())


**Metro la cotización EUR/DOL**

In [61]:
# Abrir la serie de IBEX35
df_eur_usd = pd.read_csv('datos/datos_historicos_EUR_USD.csv', parse_dates=['Fecha'], index_col='Fecha')

# Convert 'Último' column to float
df_eur_usd['Último'] = df_eur_usd['Último'].apply(lambda x: float(x.replace('.', '').replace(',', '.')))

df_eur_usd_reindexado = pd.DataFrame(index=vector_fechas, data=0, columns=df_eur_usd.columns)
df_eur_usd_reindexado = df_eur_usd_reindexado.astype(object)

# Copiar los valores de df_petroleo a df_petroleo_reindexado cuando los índices coinciden
for fecha in df_eur_usd.index:
  if fecha in df_eur_usd_reindexado.index:
    df_eur_usd_reindexado.loc[fecha] = df_eur_usd.loc[fecha]


# Min-Max Scaling
df_eur_usd_reindexado['Normalizado'] = (df_eur_usd_reindexado['Último'] - df_eur_usd_reindexado['Último'].min()) / (df_eur_usd_reindexado['Último'].max() - df_eur_usd_reindexado['Último'].min())


#REPRESENTAR LAS SERIES TEMPORALES NORMALIZADAS SOBRE EL MISMO EJE

El grafico es interactivo y permite hacer zoom.

El OJO humano y el cerebro ya ha visto que hay relación entre ambas series.

In [62]:
# 1. Select 'Último' column from both DataFrames
df_ibex_reindexado['Derivada'] = df_ibex_reindexado['Normalizado'].diff()

ibex_ultimo = df_ibex_reindexado['Normalizado']
df_petroleo_reindexado['Derivada'] = df_petroleo_reindexado['Normalizado'].diff()

petroleo_ultimo = df_petroleo_reindexado['Normalizado']
df_vix_reindexado['Derivada'] = df_vix_reindexado['Normalizado'].diff()

vix_ultimo = df_vix_reindexado['Normalizado']

df_eur_usd_reindexado['Derivada'] = df_eur_usd_reindexado['Normalizado'].diff()
eur_usd_ultimo = df_eur_usd_reindexado['Normalizado']




# Crear DataFrame para Plotly
df_to_plot = pd.DataFrame({'IBEX35': ibex_ultimo, 'Petroleo_WTI': petroleo_ultimo, 'VIX' : vix_ultimo, 'EUR_USD' : eur_usd_ultimo},index=vector_fechas)


# Crear gráfico interactivo
fig = go.Figure()

fig.add_trace(go.Scatter(x=df_to_plot.index, y=df_to_plot['IBEX35'],
                         mode='lines', name='IBEX35'))

fig.add_trace(go.Scatter(x=df_to_plot.index, y=df_to_plot['Petroleo_WTI'],
                         mode='lines', name='Petroleo_WTI'))

fig.add_trace(go.Scatter(x=df_to_plot.index, y=df_to_plot['VIX'],
                         mode='lines', name='VIX'))

fig.add_trace(go.Scatter(x=df_to_plot.index, y=df_to_plot['EUR_USD'],
                         mode='lines', name='EUR_USD'))



umbral=0.4

df_subida = df_to_plot[(df_ibex_reindexado['Derivada'] > umbral) &
                       (df_petroleo_reindexado['Derivada'] > umbral) &
                       (df_vix_reindexado['Derivada'] > umbral) &
                       (df_eur_usd_reindexado['Derivada'] > umbral)]


# Add scatter trace for df_subida
fig.add_trace(go.Scatter(
    x=df_subida.index,
    y=df_subida['IBEX35'],  # You can choose the column to represent on y-axis
    mode='markers',
    marker=dict(color='yellow', size=8),  # Customize marker appearance
    name='Subida Points'
))

# Actualizar diseño del gráfico
fig.update_layout(title='IBEX35 vs Petroleo_WTI vs VIX vs EUR_USD',
                  xaxis_title='Fecha',
                  yaxis_title='Valor',
                  hovermode='x unified',
                  template='plotly_white')

fig.show()


Yo concluiría que el índice de petroleo_WTI puede ser un indice adelantado del IBEX 35.

¿Cómo puedo calcular el tiempo de adelanto?
¿Habrá un factor de correlación?
¿Esto se podría hacer con un modelo?.
Problemas con los ejes, tienen diferente longitud, hay que ajustar siempre al eje con la misma longitud.

In [63]:
# Assuming df_to_plot is already defined and contains your data

# Set the window size for the rolling correlation
window_size = 30  # You can adjust this to your needs

# Calculate the rolling correlation
rolling_correlation = df_to_plot['IBEX35'].rolling(window=window_size).corr(df_to_plot['Petroleo_WTI'])

# Create a new DataFrame for the rolling correlation
df_correlation = pd.DataFrame({'Correlation': rolling_correlation})
df_correlation = df_correlation.reindex(df_to_plot.index, method='pad')

# You can now plot or analyze df_correlation

In [64]:
# Add trace for the correlation
fig.add_trace(go.Scatter(x=df_correlation.index, y=df_correlation['Correlation'],
                         mode='lines', name='Correlation', yaxis='y2'))

# Update layout for the second y-axis
fig.update_layout(
    yaxis2=dict(
        title='Correlation',
        overlaying='y',
        side='right',
        range=[-1, 1]  # Set range for correlation
    ),
    title='IBEX35, Petroleo_WTI, and Correlation',
    xaxis_title='Fecha',
    yaxis_title='Valor',
    hovermode='x unified',
    template='plotly_white'
)

# Show the plot

In [65]:
df_ibex_reindexado['media_movil_3'] = df_ibex_reindexado['Último'].rolling(window=3).mean()
df_ibex_reindexado['media_movil_7'] = df_ibex_reindexado['Último'].rolling(window=7).mean()
df_ibex_reindexado['media_movil_30'] = df_ibex_reindexado['Último'].rolling(window=30).mean()

df_ibex_reindexado['Señal'] = 0  # Inicializa la señal en 0

# Define la condición para la señal de compra
condicion_compra = (df_ibex_reindexado['media_movil_3'] > df_ibex_reindexado['media_movil_7']) & \
                  (df_ibex_reindexado['media_movil_7'] > df_ibex_reindexado['media_movil_30']) & \
                  (df_ibex_reindexado['media_movil_3'].shift(1) < df_ibex_reindexado['media_movil_7'].shift(1))  # Verifica el cruce

# Asigna 1 a la señal cuando se cumple la condición
df_ibex_reindexado.loc[condicion_compra, 'Señal'] = 1

caracteristicas = ['Último', 'media_movil_3', 'media_movil_7', 'media_movil_30']
X = df_ibex_reindexado[caracteristicas]
y = df_ibex_reindexado['Señal']




ENTRENAR EL MODELO

In [ ]:
# ESTE ES EL EXPERIMIENTO DE ENTRENAR UN MODELO PARA CREAR UNA SEÑAL 1/0 EN FUNCIÓN DE LAS MEDIAS MOVILES
# SEGÚN INVESTIGACIÓN, EXISTE YA UNA TÉCNICA LLAMADA MODELO ARIMA: https://en.wikipedia.org/wiki/Autoregressive_integrated_moving_average
# REHAGO EL CÓDIGO PARA ENTRENAR UN ARIMA CADA 7 DÍAS CON LOS DATOS DE 1 MES, Y QUE PREDIGA LOS VALORES DEL IBEX35 EN 7 DÍAS.

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.impute import SimpleImputer

# Divide los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

imputer = SimpleImputer(strategy='mean') # O 'median', 'most_frequent', 'constant'
X_train = imputer.fit_transform(X_train)
print(X_train)
# Crea y entrena el modelo
modelo = LogisticRegression()
modelo.fit(X_train, y_train)
import joblib

# Guardar el modelo en un fichero
joblib.dump(modelo, 'modelo_entrenado.pkl')

# Guardar el imputer en un fichero, si lo has usado
joblib.dump(imputer, 'imputer.pkl')

[[13135.4         4378.46666667  9540.54285714  7042.34      ]
 [11997.1         3999.03333333  8540.18571429  6921.14304094]
 [    0.             0.             0.          6446.29      ]
 [    0.             0.             0.          6913.43      ]
 [11840.6        11830.4         6739.91428571  6921.14304094]
 [    0.             0.          1907.17142857  6835.89666667]
 [13005.2         8608.83333333  3689.5         7774.31333333]
 [    0.             0.          1899.57142857  6921.14304094]
 [    0.             0.          5403.78571429  7010.88666667]
 [    0.             0.             0.          6921.14304094]
 [12372.8         8502.73333333  7462.87142857  6602.13666667]
 [    0.          4397.06666667  9109.85714286  6913.43      ]
 [12286.5        12145.93333333  8630.27142857  6921.14304094]
 [13484.3         8935.86666667  9553.17142857  7856.73      ]
 [    0.          4432.33333333  6676.56635338  6921.14304094]
 [13016.9         4338.96666667  5558.45714286  6163.01

['imputer.pkl']

**EVALUAR EL MODELO**

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score

# Abrir la serie de IBEX35
df_ibex_test = pd.read_csv('datos/datos_historicos_IBEX35.csv', parse_dates=['Fecha'], index_col='Fecha')
# Convert 'Último' column to float
df_ibex_test['Último'] = df_ibex_test['Último'].apply(lambda x: float(x.replace('.', '').replace(',', '.')))
# Suponiendo que 'nuevos_datos' es un DataFrame con las nuevas observaciones
nuevos_datos_reindexado = pd.DataFrame(index=vector_fechas_test, data=0, columns=df_ibex_test.columns)

# Copiar los valores de df_ibex a df_ibex_reindexado cuando los índices coinciden
for fecha in df_ibex_test.index:
  if fecha in nuevos_datos_reindexado.index:
    nuevos_datos_reindexado.loc[fecha] = df_ibex_test.loc[fecha]

nuevos_datos_reindexado['Último'] = nuevos_datos_reindexado['Último']
nuevos_datos_reindexado['media_movil_3'] = nuevos_datos_reindexado['Último'].rolling(window=3).mean()
nuevos_datos_reindexado['media_movil_7'] = nuevos_datos_reindexado['Último'].rolling(window=7).mean()
nuevos_datos_reindexado['media_movil_30'] = nuevos_datos_reindexado['Último'].rolling(window=30).mean()

nuevos_datos_reindexado = nuevos_datos_reindexado.astype(object)

datos_entrada = nuevos_datos_reindexado[caracteristicas]
datos_entrada.fillna(0, inplace=True)
datos_entrada_imputados = imputer.fit_transform(datos_entrada)

# Realiza predicciones en el conjunto de prueba
y_pred = modelo.predict(datos_entrada_imputados)

# Filter nuevos_datos_reindexado to the dates in vector_fechas_test
filtered_data = nuevos_datos_reindexado[nuevos_datos_reindexado.index.isin(vector_fechas_test)]

# Calculate the condition for intersection of moving averages
condition = (filtered_data['media_movil_3'] > filtered_data['media_movil_7']) & \
            (filtered_data['media_movil_7'] > filtered_data['media_movil_30']) & \
            (filtered_data['media_movil_3'].shift(1) < filtered_data['media_movil_7'].shift(1))

# Create y_test based on the condition
y_test = np.where(condition, 1, 0)

print(y_pred)
print(y_test)


# Calcula las métricas
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)

print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")


[0 0 0 1 0 0 0 0 0 0 0 0 1 1 1 1 1 0 0 1 1 1 1 1 0 0 1 1 1 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0]
Accuracy: 0.8118811881188119
Precision: 0.0
Recall: 0.0


<ipython-input-164-7c18e362f62e>:13: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '13.084,00' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.

<ipython-input-164-7c18e362f62e>:13: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '13.091,60' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.

<ipython-input-164-7c18e362f62e>:13: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '12.236,40' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.

<ipython-input-164-7c18e362f62e>:13: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '371,36M' has dtype incompatible wit

In [66]:
import gradio as gr
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score
import joblib

# Cargar modelo e imputador previamente entrenados
modelo = joblib.load("modelo_entrenado.pkl")
imputer = joblib.load("imputer.pkl")
caracteristicas = ["Último", "media_movil_3", "media_movil_7", "media_movil_30"]

# Función principal que ejecuta las predicciones
def predecir_archivo(archivo_csv):
    df_ibex_test = pd.read_csv(archivo_csv.name, parse_dates=['Fecha'], index_col='Fecha')

    # Convertir columna 'Último' a float
    df_ibex_test['Último'] = df_ibex_test['Último'].apply(lambda x: float(x.replace('.', '').replace(',', '.')))

    vector_fechas_test = df_ibex_test.index
    nuevos_datos_reindexado = pd.DataFrame(index=vector_fechas_test, data=0, columns=df_ibex_test.columns)

    for fecha in df_ibex_test.index:
        if fecha in nuevos_datos_reindexado.index:
            nuevos_datos_reindexado.loc[fecha] = df_ibex_test.loc[fecha]

    nuevos_datos_reindexado['media_movil_3'] = nuevos_datos_reindexado['Último'].rolling(window=3).mean()
    nuevos_datos_reindexado['media_movil_7'] = nuevos_datos_reindexado['Último'].rolling(window=7).mean()
    nuevos_datos_reindexado['media_movil_30'] = nuevos_datos_reindexado['Último'].rolling(window=30).mean()
    nuevos_datos_reindexado.fillna(0, inplace=True)

    datos_entrada = nuevos_datos_reindexado[caracteristicas]
    datos_entrada_imputados = imputer.transform(datos_entrada)

    y_pred = modelo.predict(datos_entrada_imputados)

    # Generar etiqueta real simulada (como en tu código)
    condition = (
        (nuevos_datos_reindexado['media_movil_3'] > nuevos_datos_reindexado['media_movil_7']) &
        (nuevos_datos_reindexado['media_movil_7'] > nuevos_datos_reindexado['media_movil_30']) &
        (nuevos_datos_reindexado['media_movil_3'].shift(1) < nuevos_datos_reindexado['media_movil_7'].shift(1))
    )
    y_test = np.where(condition, 1, 0)

    # Calcular métricas
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)

    # Resultado como string
    resultado = f"Accuracy: {accuracy:.2f}\nPrecision: {precision:.2f}\nRecall: {recall:.2f}"

    return resultado

# Interfaz de Gradio
iface = gr.Interface(
    fn=predecir_archivo,
    inputs=gr.File(label="Sube el CSV del IBEX35"),
    outputs="text",
    title="Predicción sobre IBEX35",
    description="Sube un archivo CSV con datos del IBEX35 y obtén las métricas del modelo entrenado."
)

iface.launch()

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://04a88cc6c56c0a3fdf.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


**FUNCIÓN DE ENTRENAIENTO DEL MODELO ARIMA. LA ENTRADA SON LAS OBSERVACIONES OBTENIDAS QUE YO CARGO DE UN FICHERO CSV. EL VECTOR_FECHAS DETERMINA QUÉ PERIODO DE VALORES SE UTILIZA PARA LA OBSERVACIÓN. *

In [67]:
import pandas as pd
from sklearn.model_selection import train_test_split
from statsmodels.tsa.arima.model import ARIMA
import joblib

def train_arima_model():
  #LOS VALORES DEL IBEX EN LAS FECHAS DADAS ESTÁN EN df_ibex_reindexado
  # Filter data based on user input
  # Use .loc to slice by labels, handling potential missing dates
  filtered_df = df_ibex_reindexado
  filtered_df['Último'] = filtered_df['Último'].astype(float)  # Ensure numeric type
  #filtered_df['Último'].fillna(filtered_df['Último'].mean(), inplace=True)  # Replace NaN with mean
  filtered_df = df_ibex_reindexado.loc[df_ibex_reindexado['Último'] != 0]
  # Prepare the data for training
  y = filtered_df['Último'].values  # Target: IBEX35 values

  # Split the data into training and testing sets
  y_train, y_test = train_test_split(y, test_size=0.2, shuffle=False)

  # Determine the ARIMA order (p, d, q) - you might need to experiment with these values
  # You can use tools like ACF and PACF plots to help determine the order
  # Example order: (5, 1, 0)
  # https://en.wikipedia.org/wiki/Autoregressive_integrated_moving_average


  order = (5, 1, 0)


  # Create and train the ARIMA model
  model = ARIMA(y_train, order=order)
  model_fit = model.fit()

  # Save the trained model
  joblib.dump(model_fit, 'ibex35_arima_model.pkl')

  print("ARIMA model trained and saved successfully!")



**FUNCION DE PREDICCIÓN DE VALORES DEL IBEX A PARTIR DE LAS ÚLTIMAS 30 OBSERVAICONES**


In [70]:
# prompt: quiero crear una aplicación de gradio que use el modelo ibex35_arima recien entrenado para predecir los 7 valores siguientes del IBEX35

import joblib
import pandas as pd
import numpy as np
import gradio as gr
from statsmodels.tsa.arima.model import ARIMA


def predict_ibex35(df_last_observations,days_to_predict=7):

    # Obtener las últimas observaciones del conjunto de datos (reemplaza con tu DataFrame)
    # Asegúrate de que las fechas en tu DataFrame sean de tipo datetime64[ns]
    # y estén ordenadas cronológicamente.
    # Ejemplo, asumimos que df_ibex_reindexado ya está definido y cargado con datos:
    # Eliminar ceros
    #df_ibex_reindexado_filtered = df_ibex_reindexado[df_ibex_reindexado['Último'] != 0]
    last_observations = df_last_observations.astype(float)  # Obtener las ultimas 30 observaciones

    # Realizar predicciones con el modelo ARIMA cargado
    predictions = []
    for i in range(days_to_predict):
        # Extender la serie temporal con la predicción anterior
        model_fit = ARIMA(last_observations.values, order=(5, 1, 0)).fit()  # Ajustar con las ultimas 30 obs.
        prediction = model_fit.predict(start=len(last_observations), end=len(last_observations)) # Predice el prox valor
        predictions.append(prediction[0])
        last_observations = pd.concat([last_observations, pd.Series([prediction[0]])], ignore_index=True)
        # Convert to list of regular Python floats using tolist()
        predictions = np.array(predictions).tolist()
    return predictions



import schedule
import time
import pandas as pd

def initialize_app():
    """
    Inicializa la aplicación obteniendo los últimos 30 valores del IBEX35 del archivo CSV.
    Lanza la tarea periódica si no está en ejecución para crear y actualizar el archivo.

    Returns:
        pandas.DataFrame: Un DataFrame con las últimas 30 observaciones del IBEX35 o
                         un DataFrame vacío si hay un error.
    """
    try:
        # Iniciar la tarea programada si no está en ejecución
        if not schedule.get_jobs():
            schedule.every().day.at("00:00").do(descargar_y_guardar_ibex35)
            print("Tarea programada iniciada para descargar datos del IBEX35.")
        else:
            print("La tarea programada ya está en ejecución.")

        # Ejecutar tareas pendientes de schedule
        schedule.run_pending()

        # Leer los datos del archivo CSV
        df_ibex35 = pd.read_csv('./datos/ibex35_data.csv', index_col='Date')

        # Asegurarse de que el índice sea de tipo datetime
        df_ibex35.index = pd.to_datetime(df_ibex35.index)

        # Obtener las últimas 30 observaciones
        valores = df_ibex35['^IBEX'].tail(30)
        return valores

    except FileNotFoundError:
        print("El archivo ibex35_data.csv no se ha encontrado. "
              "Asegúrate de que la tarea periódica se ha ejecutado al menos una vez.")
        return pd.DataFrame()  # Devolver un DataFrame vacío en caso de error
    except Exception as e:
        print(f"Error al inicializar la aplicación: {e}")
        return pd.DataFrame()  # Devolver un DataFrame vacío en caso de error


def launch_gradio():
    # Inicializar la aplicación
    initial_values = initialize_app()

    with gr.Blocks() as demo:
        with gr.Tab("Predict IBEX35"):
            days_to_predict_input = gr.Number(label="Número de días a predecir", value=7)
            predict_button = gr.Button("Predict")
            predict_output = gr.Textbox(label="Predicciones del IBEX35")

            # Predecir utilizando el modelo entrenado
            predict_button.click(
                fn=predict_ibex35,
                inputs=[gr.State(initial_values), days_to_predict_input],
                outputs=predict_output,
            )

    demo.launch(share=True)
#descargar_y_guardar_ibex35()
launch_gradio()
#initial_values = initialize_app()
#train_arima_model()
#print("La predicción de 7 días para el IBEX35, a partir del modelo entrenado,es:")
#predict_ibex35(initial_values,7)


La tarea programada ya está en ejecución.
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://4e6cbe6483774a9a5d.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [18]:
import yfinance as yf
import pandas as pd

def obtener_ultimos_valores_ibex35_internet(ticker="^IBEX", num_valores=30):
    """
    Obtiene los últimos valores del IBEX35 y sus fechas desde Yahoo Finance
    y los devuelve como un DataFrame.

    Args:
        ticker (str, optional): El ticker del IBEX35 en Yahoo Finance.
                                 Por defecto es "^IBEX".
        num_valores (int, optional): Número de últimos valores a obtener.
                                      Por defecto es 30.

    Returns:
        pandas.DataFrame: Un DataFrame con los valores del IBEX35 y las fechas
                         como índice.
    """
    try:
        # Descargar datos del IBEX35 desde Yahoo Finance
        ibex_data = yf.download(ticker, period=f"{num_valores}d")

        return ibex_data['Close']

    except Exception as e:
        print(f"Error al obtener datos de Yahoo Finance: {e}")
        return pd.DataFrame()  # Devolver un DataFrame vacío en caso de error

In [23]:
import schedule
import time
import pandas as pd  # Si aún no la has importado
from google.colab import drive # Para guardar en Google Drive

def descargar_y_guardar_ibex35():
  """Descarga los últimos 30 valores del IBEX35 y los guarda en un archivo CSV en Google Drive."""

  # Descarga los datos
  ibex_data = obtener_ultimos_valores_ibex35_internet(ticker="^IBEX", num_valores=30)

  # Guarda los datos en un archivo CSV en Google Drive
  ruta_archivo = './datos/ibex35_data.csv' # Cambia la ruta si lo necesitas
  ibex_data.to_csv(ruta_archivo, index=True)  # index=True para guardar las fechas

  print(f"Datos del IBEX35 guardados en: {ruta_archivo}")

In [20]:
# Programa la tarea para que se ejecute todos los días a las 00:00
schedule.every().day.at("00:00").do(descargar_y_guardar_ibex35)

# Mantén el bucle while True para que la tarea se ejecute indefinidamente
while True:
  schedule.run_pending()
  time.sleep(1)  # Espera 1 segundo antes de la siguiente verificación

KeyboardInterrupt: 